# Preference Tuning for Summarization using Synthetic Data

**⏱️ Time to complete**: 10 hours

Preference tuning is a powerful tool that can optimize LLMs towards complex preferences that cannot be easily captured through supervised fine-tuning. However, manually annotating preferences between model outputs using human raters can be extremely time-consuming and expensive. Instead, synthetic preference data can be generated by scoring responses with large foundation models, allowing for much cheaper and scalable data collection!

Here we'll go through an end-to-end example for preference tuning of an open-source language model with synthetic data, covering scalable methodologies for data preprocessing, fine-tuning and evaluation, using Ray. We will focus on the task of summarization for the [CNN/DailyMail](https://huggingface.co/datasets/abisee/cnn_dailymail) dataset. 

This notebook is based on the following blog post: `TODO`.

Notebook guide:
- 🔄 REPLACE indicates to replace with your unique values
- 💡 INSIGHT indicates infrastructure insight

# Table of Contents
1. [Data Preprocessing](#step-1-data-preprocessing): In this section we cover how we can prepare preference data for the summarization task using an LLM-as-a-judge. 
    1. [Generate Multiple Choice Questions From Articles](#part-a-generate-multiple-choice-questions-from-articles)
    2. [Generate Summaries and Scores](#part-b-generate-summaries--scores)
    3. [Generate Preference Tuning Data](#part-c-generate-preference-tuning-data)
2. [DPO Finetuning](#step-2-fine-tuning): This section will cover how you can fine-tune an open source model on the preference data on the Anyscale platform.
3. [Evaluation](#step-3-evaluation): The section will lay down a blue-print for evaluation and compare performance to that of closed source models like OpenAI's GPT-4.

First, let's make the necessary imports

In [1]:
import os
import pprint
import textwrap

import ray.data
import datasets


from src.utils.models import DataSchema
from src.utils.common import print_wrapped

os.environ["PYTHONPATH"] = f"{os.environ.get('PYTHONPATH', '')}:src"

# Step 1: Synthetic Data Generation

First, let's inspect the training dataset and look at an example. 

In [ ]:
hf_ds = datasets.load_dataset("abisee/cnn_dailymail", "3.0.0", split="train").shuffle(
    seed=21
)
# extract a subset of 20000 articles
hf_ds_subset = hf_ds.select(range(20000))

ray_ds = ray.data.from_huggingface(hf_ds_subset)
raw_example = ray_ds.take(1)[0]

(ReadParquet pid=2764, ip=10.0.30.152) Traceback (most recent call last):
(ReadParquet pid=2764, ip=10.0.30.152)   File "pyarrow/public-api.pxi", line 128, in pyarrow.lib.pyarrow_wrap_data_type
(ReadParquet pid=2764, ip=10.0.30.152)   File "pyarrow/types.pxi", line 508, in pyarrow.lib.ListType.init
(ReadParquet pid=2764, ip=10.0.30.152)   File "pyarrow/types.pxi", line 220, in pyarrow.lib.DataType.init
(ReadParquet pid=2764, ip=10.0.30.152)   File "pyarrow/types.pxi", line 94, in pyarrow.lib._datatype_to_pep3118
(ReadParquet pid=2764, ip=10.0.30.152)   File "/home/ray/anaconda3/lib/python3.11/site-packages/ray/air/util/tensor_extensions/arrow.py", line 142, in __arrow_ext_deserialize__
(ReadParquet pid=2764, ip=10.0.30.152)     @classmethod
(ReadParquet pid=2764, ip=10.0.30.152) 
(ReadParquet pid=2764, ip=10.0.30.152) KeyboardInterrupt: 
(ReadParquet pid=2947, ip=10.0.30.152) 
(ReadParquet pid=2944, ip=10.0.30.152) 
(ReadParquet pid=2954, ip=10.0.30.152) 
(ReadParquet pid=2950, ip=10.0

(autoscaler +1h21m12s) [autoscaler] Downscaling node i-0b17ff1a8403dae9e (node IP: 10.0.30.152) due to node idle termination.
(autoscaler +1h25m49s) [autoscaler] [48CPU-192GB] Upscaling 1 node(s).
(autoscaler +1h25m50s) [autoscaler] [48CPU-192GB|m5.12xlarge] [us-west-2a] [on-demand] Launched 1 instances.
(autoscaler +1h29m19s) [autoscaler] Downscaling node i-0e6302ce30c813623 (node IP: 10.0.58.223) due to node idle termination.


In [3]:
pprint.pprint(raw_example, width=80)

{'article': 'Scam: Lisa Harrison, 34, promised customers low currency rates on '
            'US dollars and special deals . A wedding planner who stole '
            "£80,000 from couples in a bid to satisfy an 'out-of-control' "
            'online gambling addiction has been jailed. Lisa Harrison, 34, '
            'began taking money from her clients in summer 2013 by enticing '
            'them with low currency rates on US dollars and flight upgrades. '
            'She took money from 19 couples who had entrusted their savings to '
            'her after being promised the wedding of their dreams. It is '
            'understood that the company she worked for, iPlan New York, '
            'specialised in weddings in New York City. Her website '
            "iplannewyork.com, which has been taken down, said: 'iPlan New "
            'York was set up to create and style the perfect tailor made '
            "wedding for couples travelling to New York to get married! 'We "
     

Look at the example article. Our goal is to summarize it. Where do we get the data from? 
We can use the summarizer LLM to generate candidate summaries. How do we score them? In this example, we will employ a _synthetic_ summary scoring method using another LLM as a judge. We score the correctness of a summary using the following metrics:

**Summary Scoring Metrics**
1. Multiple choice Q&A accuracy:
    - Given the original text, we use an LLM judge to generate 5 multiple choice questions about the text.
    - We then ask the LLM judge to answer the questions using only the summary, and record the number of questions correctly answered.
2. Word count: We simply count the number of words in the summary.

This allows us to construct a simple preference function between two summaries:

**Preference Function**
1. If both summary responses attain more than 3/5 multiple choice questions correct, we will prefer the shorter response. We do not care about Q&A accuracy beyond 3 correct answers, since the summary should not contain all information from the text.
2. Otherwise, we select the response that leads to more correctly answered multiple choice questions.

To generate the preference pairs, we will generate 10 summaries from each article using the model we wish to fine-tune. Then, we will randomly sample pairs of summaries and use our preference function to annotate the preference between them.

For this example, we will use `Mistral-7B-Instruct-v0.1` as the base model to fine-tune and `Llama-3.1-70B-Instruct` as a judge. Note that mistral-instruct is already instruction tuned, so that given a prompt to do summarization it might do a good job, but it may not be aligned with how we want the summarization to look like. We can use preference data to further align the instruct variant towards our specific needs.

Combining all this together, our data pre-processing pipeline is going to look as follows: 

![preprocessing](./assets/preprocessing.png?1)

### Part (a): Generate Multiple Choice Questions from Articles

First, we will generate the multiple choice questions and answers for each article using `Llama-3.1-8B-Instruct` (or `70B` if you have A100/H100s). Leveraging vLLM and Ray, we can very easily scale this generation process across multiple GPUs.


>  **_NOTE:_**  We provide two sets of configs: One with an 8B parameter model as the judge, and another with the 70B model. Using the 8B model is recommended for quicker runtimes, since we make use of highly available A10Gs. For good performance, and to replicate the results in our blog, you should use the 70B judge model which uses A100s. 

The following command will run the [src/scripts/generate_questions.py](./src/scripts/generate_questions.py) script, which generates the questions and answers and saves them in `.parquet` files.

This step will take ~75 min for 8B running on A10s and ~?? for 70B running on A100s.

💡 INSIGHT  
We are running this script as an anyscale job. The resources required by each step are requested at runtime and provisioned by Anyscale's autoscaler based on availability and quotas. You can change the [qa_generation](./configs/qa_generation) however you want. Most important parameters regarding resources are `accelerator_type`, `num_gpus_per_instance`, and `concurrency`. This script will generate 5 multiple choice question and answer pairs per article for 21k examples. According to the [llama_8b](./configs/qa_generation/llama_8b.yaml) config we are requesting 3 replicas of 4xA10G machines processing a batch-size of 128 examples each which saturates the GPUs all the way through.

In [ ]:
!anyscale job submit -f configs/jobs/8b_judge/generate_questions_job.yaml
# Optional: use the 70b model for better performance (runs on A100s)
# !anyscale job submit -f configs/jobs/70b_judge/generate_questions_job.yaml

At the end of the job, you should see the remote path to the folder with Q&A in the logs.


<p align="center">
  <img src="./assets/question_generation_done.png?" alt="Evaluation" width=800>
</p>

 Make sure to make note to use it for the next steps! 

 🔄 REPLACE the resulting s3 url here. If you want to skip the prior step, you can continue with the prepared example data below.

In [ ]:
# Replace this with the link to the output folder from the previous job
qa_folder = "s3://air-example-data/preference-tuning-summarization-example/qa_generation/qa_annotations_full_train/"
qa_ds = ray.data.read_parquet(qa_folder)
# The dataset is small, we can materalize it
example_rows = qa_ds.materialize().take(3)

In [7]:
for row in example_rows:
    print_wrapped("TEXT", row[DataSchema.ARTICLE])
    print_wrapped("QUESTIONS", row[DataSchema.MCQ_QUESTIONS])
    print_wrapped("ANSWERS", str(row[DataSchema.GROUND_TRUTH_MCQ_ANSWERS]))
    pprint.pprint("=" * 80, width=80)

TEXT:
From balloon-popping lasers to Wolverine-style claws, there are numerous concept
and protoype weapons designed by wannabe superhero inventors. But, a magician
has not only created a wristband that turns the wearer into Pyro from the Marvel
comics, he is selling it for $174 (£111) online. Named after the comic book
mutant, the Pyro band features four chambers that fires four fireballs, and it
can be controlled from the wrist or remotely. Scroll down for video . Pyro
(pictured) was designed by New Hampshire magician Adam Wilber. It features four
separate chambers for four multiple shots and can be controlled either from the
wrist or remotely . Its inventor, Adam Wilber explained: ‘Fire. Since the dawn
of time it has been the reward at the end of man's quest. Both creator and
destroyer, it has historically been the element hardest to control. ‘Until now.
Your quest is over. The power of fire in the palm of your hand. That's the power
of Pyro.' It is available from the Ellusionist si

### Part (b): Generate Summaries + Scores

Next, we will generate 10 summaries for each article in the training set and score them with our Q&A judging setup. 

The following command will run the [generate_summaries_and_scores.py](src/scripts/generate_summaries_and_scores.py) script, which takes in the folder with generated questions + articles and stores the results to a new folder of `.parquet` files. This script will use the model under training to produce 10 summaries per each example on all of the input data examples. Followed by each summarization, it will also perform summary accuracy measurement, asking the down-stream LLM to answer the questions generated earlier solely based on the summaries generated by the desired model. 

This job will take ~?? min for 8B and ~?? min for 70B given the default configurations.

In [ ]:
!anyscale job submit -f configs/jobs/8b_judge/generate_summaries_train_job.yaml 
# Optional: use the 70b model for better performance (runs on A100s)
# !anyscale job submit -f configs/jobs/70b_judge/generate_summaries_train_job.yaml

In [ ]:
# replace with the link to the generated summaries
summary_folder = "s3://air-example-data/preference-tuning-summarization-example/summary_generation_base/train/"
summary_ds = ray.data.read_parquet(summary_folder)
example_rows = summary_ds.take(1)

In [10]:
from src.utils.models import DataSchema

for row in example_rows:
    print_wrapped("TEXT", row[DataSchema.ARTICLE])
    print_wrapped("QUESTIONS", row[DataSchema.MCQ_QUESTIONS])
    print_wrapped("MODEL GENERATED SUMMARY", row[DataSchema.SUMMARY_GENERATION_RAW_OUTPUT])
    print_wrapped("ANSWERS", str(row[DataSchema.GROUND_TRUTH_MCQ_ANSWERS]))
    print_wrapped("JUDGE ANSWERS FROM SUMMARY", str(row[DataSchema.JUDGE_MCQ_ANSWERS]))
    pprint.pprint("=" * 100, width=80)

TEXT:
(RollingStone.com) -- Jennifer Lawrence, the 20-year-old Oscar nominee for Best
Actress, is sitting in a fancy Manhattan hotel sipping tea and feeling a little
out of place. See, she grew up in Louisville, Kentucky, where her dad owned a
construction company and her mom ran a summer camp. They had land and horses.
She loved to fish. She was a total tomboy: field hockey, softball, basketball on
an all-boys team. ("I was so dykey.") One of her nicknames was Nitro. She lives
in Los Angeles now, but "little redneck things still come out." Like what? "I'm
attracted to my brother. Stuff like that." 10 Best Movies of 2010 . At 14, she
decided she wanted to be an actress and dragged her mom to New York for
auditions. The people at Reese's Peanut Butter Cups told her she was the best
they'd ever seen. Her mom told her they were lying. (Her mom didn't like showbiz
much.) She auditioned for the role of Bella in "Twilight," which would have been
perfect if Bella were a badass, but since she'

### Part (c): Generate Preference Tuning Data

Next, we will generate 10 summaries for each article in the training set and score them with our Q&A judging setup. 

The following command will run the [generate_dpo_data.py](src/scripts/generate_dpo_data.py) script, which takes in the folder of summaries and outputs `.jsonl` files for training and validation.

In [ ]:
!python src/scripts/generate_dpo_data.py configs/training_data_generation/mistral_8b.yaml

In [10]:
# Inspect the results
# Replace with the link to your validation file
validation_file = "s3://air-example-data/preference-tuning-summarization-example/dpo_training_data/valid.jsonl"

valid_ds = ray.data.read_json(validation_file)
example_rows = valid_ds.take(1)

2024-08-16 15:28:20,813	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-08-16_13-49-43_360011_2410/logs/ray-data
2024-08-16 15:28:20,814	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ExpandPaths] -> TaskPoolMapOperator[ReadFiles] -> LimitOperator[limit=1]


- ExpandPaths 1:   0%|          | 0/1 [00:00<?, ?it/s]

- ReadFiles 2:   0%|          | 0/1 [00:00<?, ?it/s]

- limit=1 3:   0%|          | 0/1 [00:00<?, ?it/s]

Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
for row in example_rows:
    print_wrapped("PROMPT", row["chosen"][0]["content"])
    print_wrapped("CHOSEN RESPONSE", row["chosen"][1]["content"])
    print_wrapped("REJECTED RESPONSE", row["rejected"][1]["content"])

PROMPT:
Given the following text, create a very short summary that is at most 2
sentences.  Text: By . Tamara Cohen, Political Reporter . PUBLISHED: . 18:32
EST, 27 January 2013 . | . UPDATED: . 08:48 EST, 28 January 2013 . Deputy Prime
Minister Nick Clegg and his wife Miriam are determined to keep the education of
their 11-year-old son 'out of politics' Nick Clegg yesterday defended the
possibility he may send his children to private schools as it emerged he and his
wife Miriam have not even visited their local state school. He said the
education of his 11-year-old son Antonio, who starts secondary school this year,
should not be used as 'a political football' and that the couple would do
'what's best' for their children although he was braced for criticism. Last week
the Liberal Democrat leader told listeners to his radio show he would send his
son to a private school if he failed to find a place in a good comprehensive,
saying he would use the state system 'if it works out', but tha

# Step 2: Fine-tuning

Now that we have the pre-processed dataset, we are ready to fine-tune `Mistral-7B-Instruct-v0.1` using DPO. On Anyscale, we've created an easy-to-use interface to do preference-tuning using `DPO`. We leverage Ray to overlap reference model log-probability calculation with model training to improve GPU utilization. Most implementations compute log probabilities synchronously with model training,

![hf model](assets/hf_dpo.png)

While our implementation using Ray is asynchronous:  


![assistant model](assets/anyscale_dpo.png)

Further, our use of Ray Data also implies that the compute configuration for the reference model can be completely decoupled with the policy model. For example, reference model calculation can run on a different node (with configurable number of GPUs, etc) with zero code changes needed. 


To get started with DPO training, we provide the config for DPO in [configs/mistral_dpo_summarization.yaml](configs/mistral_dpo_summarization.yaml) . 

In [ ]:
!cat configs/mistral_dpo_summarization.yaml

You can run the below command from the root directory for the template (`~/default`): 

```
llmforge anyscale finetune end-to-end-examples/fine-tune-preference/configs/mistral_dpo_summarization.yaml
```

# Step 3: Evaluation

Let's evaluate our trained model. Here we'll use two baselines: (1) the base model before finetuning (reference model in DPO) and (2) GPT-4o.

## Evaluation strategy

Our evaluation strategy involves the same Q&A scoring system as used while generating the preference data. 

<p align="center">
  <img src="./assets/eval.png?" alt="Evaluation" width=800>
</p>

We evaluate the baseline model and the trained DPO model on the test set. 

## Obtain summaries on the test set
First, we'll need to obtain the summaries (and scores) for both the models on the given test set. 

For the baseline model, you can simply run the below command:

In [ ]:
!anyscale job submit -f configs/jobs/8b_judge/generate_summaries_eval_baseline_job.yaml
# Optional: use the 70b model for better performance (runs on A100s)
# !anyscale job submit -f configs/jobs/70b_judge/generate_summaries_eval_baseline_job.yaml

For the fine-tuned DPO model, we provide a dummy config in [configs/summary_generation/8b_judge/mistral_finetuned_eval.yaml](configs/summary_generation/8b_judge/mistral_finetuned_eval.yaml). If you used the default training config provided, the model would be trained using LoRA and you should have a path to the LoRA weights.

In [15]:
!cat configs/summary_generation/8b_judge/mistral_finetuned_eval.yaml

mode: eval
input_folder: s3://air-example-data/preference-tuning-summarization-example/qa_generation/qa_annotations_full_test
inference_type: offline
model_inference_config:
  model_id_or_path: mistralai/Mistral-7B-Instruct-v0.1 # <---- Modify with s3 link to full param weights if you did full-param training
  adapter_id_or_path: <lora_path_here> # <---  Add path to lora weights here. If you did full param training, you can instead remove this field.
  temperature: 0
  top_p: 0.95
  scaling_config:
    batch_size: 64
    concurrency: 2
    num_gpus_per_instance: 1
    accelerator_type: A10G
num_generations: 1
judge_inference_config:
  model_id_or_path: meta-llama/Meta-Llama-3.1-8B-Instruct
  temperature: 0
  scaling_config:
    batch_size: 64
    concurrency: 3
    num_gpus_per_instance: 2
    accelerator_type: A10G
num_mcq_questions: 5


In [ ]:
!anyscale job submit -f configs/jobs/8b_judge/generate_summaries_eval_finetuned_job.yaml
# Optional: use the 70b model for better performance (runs on A100s)
# !anyscale job submit -f configs/jobs/70b_judge/generate_summaries_eval_finetuned_job.yaml

In the logs for the above jobs, you should see the final path to the output summaries for both the models. 

Optionally, you can also obtain the summaries and scores for the `gpt-4o` model from OpenAI. Simply run:

In [ ]:
!anyscale job submit -f configs/jobs/8b_judge/generate_summaries_eval_gpt_job.yaml
# Optional: use the 70b model for better performance (runs on A100s)
# !anyscale job submit -f configs/jobs/70b_judge/generate_summaries_eval_gpt_job.yaml

## Get Evaluation Statistics

We've provided a convenient script [get_eval_stats.py](src/scripts/get_eval_stats.py) to get evaluation statistics and obtain the "win rate" of the DPO model (the percentage of times the DPO model performs better than the baseline). We've provided an example configuration below. 

In [ ]:
# make sure to substitute -outputs-path with your path
!python src/scripts/get_eval_stats.py --outputs-path s3://air-example-data/preference-tuning-summarization-example/summary_generation_dpo_model/test/ --baseline-outputs-path s3://air-example-data/preference-tuning-summarization-example/summary_generation_base/test/  

# (Optional): if you obtained results for GPT-4o, you should uncomment and run the following command instead
# !python src/scripts/get_eval_stats.py --outputs-path s3://air-example-data/preference-tuning-summarization-example/summary_generation_dpo_model/test/ --baseline-outputs-path s3://air-example-data/preference-tuning-summarization-example/summary_generation_base/test/  --gpt4o-outputs-path <add-path-to-gpt4o-results>

You should see the following results for the 70B model:

```text 
╒═════════════════════════════╤═══════════╤════════════╤═══════════╕
│           Metric            │   Model   │  Baseline  │  GPT-4o   │
╞═════════════════════════════╪═══════════╪════════════╪═══════════╡
│        Accuracy >=3         │ 65.4286 % │ 43.0476 %  │ 37.2381 % │
├─────────────────────────────┼───────────┼────────────┼───────────┤
│        Accuracy >=4         │ 25.7143 % │ 13.5238 %  │ 10.0000 % │
├─────────────────────────────┼───────────┼────────────┼───────────┤
│     Median Compression      │ 11.5794 % │ 12.7316 %  │ 8.0496 %  │
├─────────────────────────────┼───────────┼────────────┼───────────┤
│      Mean Compression       │ 13.0029 % │ 14.3444 %  │ 9.3554 %  │
├─────────────────────────────┼───────────┼────────────┼───────────┤
│      Summary Too Long       │ 0.0000 %  │  0.0000 %  │ 0.0000 %  │
├─────────────────────────────┼───────────┼────────────┼───────────┤
│ Contains Invalid Characters │ 0.0000 %  │  0.0952 %  │ 0.0000 %  │
╘═════════════════════════════╧═══════════╧════════════╧═══════════╛


Model Win Rate against Baseline: 74.0000 %
GPT-4o Win Rate against Baseline: 64.8095 %
```

Our fine-tuned model is able to generate much better summaries, that are more concise (compression ratio is lower) with lesser out-of-distribution characters (gibberish tokens) than the baseline. You can see more details on the same in our blog!

| **NOTE:** The evaluation results will differ if you used the 8B model which is less capable as a LLM-judge. 

## Summary

Congrats! You have now fine-tuned an open source model on preference data. As a quick recap, here's what we demonstrated in this notebook:
1. Synthetically generating preference data for DPO 
2. DPO fine-tuning of a language model on the Anyscale Platform
4. Evaluating the model against the baseline and GPT-4o, and analysing the results.